In [2]:
import pandas as pd
import os
import pickle

from sklearn.metrics import f1_score

# Data and model imports

In [3]:
path = os.getcwd() + '/data'
X_test = pd.read_csv(path + '/X_test.csv')
y_test = pd.read_csv(path + '/y_test.csv')

In [4]:
with open('models/under_lr.pkl', 'rb') as f:
    u_lr = pickle.load(f)
f.close()

with open('models/over_lr.pkl', 'rb') as f:
    o_lr = pickle.load(f)
f.close()

# Evaluation

In [5]:
under_y_pred = u_lr.predict(X_test)
over_y_pred = o_lr.predict(X_test)

/Users/jakubkozlowski/anaconda3/envs/data_mining/lib/python3.12/site-packages/sklearn/preprocessing/_encoders.py:227: UserWarning: Found unknown categories in columns [2, 3, 4] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [6]:
print(f1_score(y_test, over_y_pred))

0.4378238341968912
